# Where to Open a Coffee Shop in Los Angeles (LA)

## Introduction

_Opening a coffee shop, or any restaurant for that matter, can be extremely difficult - especially during the current pandemic. One of the most difficult and critical choices of any business owner is locating their shop. In the restaurant business a good location is often a significant factor in the ability of the restaurant to make it and suceed. The goal of this project and analysis is to determine what locations within the LA county a potentially offer a business owner the ideal location for a coffee shop targeting working adults_

This "ideal" location will be defined by a few critical factors: 
* Proximity to other coffee shops
* Proximity to other restaurants 
* Proximity to offices to target working adults

## Data Sources

In order to complete this analysis some critical data needs to be located:
* Location of potential shops that likely serve coffee in LA (ex. coffee roaster or cafe)
* Location of coffee shops in LA
* Location of offices in LA

To get this data the follow sources will need to be used:

* Foursquare API to help provide the locations of various restaurants including their type (i.e. coffee shop)
* Foursquare API to help provide the locations of various offices 
* List of neighborhoods in LA which has been made available by UCLA

This data will allow us to compare the various parts of LA to determine which neighborhood will be an ideal cadidate for a coffee shop targeting working adults

## Data Gathering

In this section we will gather the data necessary for analysis. I will first load the Atlanta neighborhood data and then gather the necessary info from the Foursquare API

In [34]:
!pip install geojson
!pip install folium
!pip install geopandas
!pip install turfpy
!pip install shapely

In [66]:
#gather the info for the Atlanta neighborhoods 
import geopandas as gpd
import turfpy
import pandas as pd
import numpy as np
from turfpy.measurement import centroid
import requests
from shapely.geometry import shape
from geojson import Polygon


la_data = 'http://gis.ucla.edu/geodata/dataset/93d71e41-6196-4ecb-9ddd-15f1a4a7630c/resource/6cde4e9e-307c-477d-9089-cae9484c8bc1/download/la-county-neighborhoods-v6.geojson'
la_df = gpd.read_file(la_data)

la_df['Latitude'] = la_df['geometry'].centroid.y
la_df['Longitude'] = la_df['geometry'].centroid.x

la_df.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



,kind,external_id,name,slug,set,metadata,resource_uri,geometry,Latitude,Longitude
0,L.A. County Neighborhood (V6),acton,Acton,acton-la-county-neighborhood-v6,/1.0/boundary-set/la-county-neighborhoods-v6/,"{'city': '', 'name': 'Acton', 'region': 'antel...",/1.0/boundary/acton-la-county-neighborhood-v6/,"MULTIPOLYGON (((-118.20703 34.53902, -118.1894...",34.495582,-118.185759
1,L.A. County Neighborhood (V6),adams-normandie,Adams-Normandie,adams-normandie-la-county-neighborhood-v6,/1.0/boundary-set/la-county-neighborhoods-v6/,"{'city': 'los-angeles', 'name': 'Adams-Normand...",/1.0/boundary/adams-normandie-la-county-neighb...,"MULTIPOLYGON (((-118.30800 34.03740, -118.3006...",34.031363,-118.300314
2,L.A. County Neighborhood (V6),agoura-hills,Agoura Hills,agoura-hills-la-county-neighborhood-v6,/1.0/boundary-set/la-county-neighborhoods-v6/,"{'city': '', 'name': 'Agoura Hills', 'region':...",/1.0/boundary/agoura-hills-la-county-neighborh...,"MULTIPOLYGON (((-118.77621 34.16816, -118.7263...",34.151085,-118.760563
3,L.A. County Neighborhood (V6),agua-dulce,Agua Dulce,agua-dulce-la-county-neighborhood-v6,/1.0/boundary-set/la-county-neighborhoods-v6/,"{'city': '', 'name': 'Agua Dulce', 'region': '...",/1.0/boundary/agua-dulce-la-county-neighborhoo...,"MULTIPOLYGON (((-118.37822 34.48811, -118.3783...",34.510666,-118.312386
4,L.A. County Neighborhood (V6),alhambra,Alhambra,alhambra-la-county-neighborhood-v6,/1.0/boundary-set/la-county-neighborhoods-v6/,"{'city': '', 'name': 'Alhambra', 'region': 'sa...",/1.0/boundary/alhambra-la-county-neighborhood-v6/,"MULTIPOLYGON (((-118.12175 34.10504, -118.1168...",34.083994,-118.135471


In [123]:
CLIENT_ID = 'ESW2O5H4KM2FXQJPDNTOESDLCIDK5ETP2JFT4EIHJAWZ1JDG'
CLIENT_SECRET = 'DLT5QD1AC5QCSH5IJZ4D1U42XV3HYKPL1CN5KLKXCSRSLGND' 
VERSION = '20210124'

import json

def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT = 50):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name) 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items'] 
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']      
        except:
            venues_list = []
    return(nearby_venues)

la_places = getNearbyVenues(names=la_df['name'],
                                   latitudes=la_df['Latitude'],
                                   longitudes=la_df['Longitude']
                                  )
print(la_places)

Acton
Adams-Normandie
Agoura Hills
Agua Dulce
Alhambra
Aliso and Wood Regional Park
Aliso Viejo
Alondra Park
Altadena
Anaheim
Angeles Crest
Arcadia
Arleta
Arlington Heights
Artesia
Athens
Atwater Village
Avalon
Avocado Heights
Azusa
Baldwin Hills/Crenshaw
Baldwin Park
Bel-Air
Bellflower
Bell Gardens
Bell
Beverly Crest
Beverly Grove
Beverly Hills
Beverlywood
Boyle Heights
Bradbury
Brea
Brentwood
Broadway-Manchester
Buena Park
Burbank
Calabasas
Canoga Park
Carson
Carthay
Castaic Canyons
Castaic
Central-Alameda
Century City
Cerritos
Charter Oak
Chatsworth
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Citrus
Claremont
Cleveland National Forest
Commerce
Compton
Costa Mesa
Coto de Caza
Covina
Cudahy
Culver City
Cypress
Cypress Park
Dana Point
Del Aire
Del Rey
Desert View Highlands
Diamond Bar
Downey
Downtown
Duarte
Eagle Rock
East Compton
East Hollywood
East Los Angeles
East Pasadena
East San Gabriel
East Whittier
Echo Park
Elizabeth Lake
El Monte
El Segundo
El Sereno
Elys

In [122]:
la_places['Venue Category'] = la_places['Venue Category'].astype('string')
la_places.dtypes

Neighborhood               object
Neighborhood Latitude     float64
Neighborhood Longitude    float64
Venue                      object
Venue Latitude            float64
Venue Longitude           float64
Venue Category             string
dtype: object

## Visual Analysis

In [82]:
import folium

#map Atlanta along with the respective neighborhoods 
la_map = folium.Map(location=[34.052235,  -118.243683], zoom_start=9)

for lat, lon in zip(atl_df['Latitude'], atl_df['Longitude']):
   folium.Marker([lat, lon], color='red', fill=True, radius=10000, parse_html=True).add_to(la_map)

folium.GeoJson(la_df['geometry']).add_to(la_map)


la_map